In [2]:
from __future__ import absolute_import, division, print_function

from copy import deepcopy
import json
import glob
import os
import time

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd


def calc_iou_individual(pred_box, gt_box):
    """Calculate IoU of single predicted and ground truth box

    Args:
        pred_box (list of floats): location of predicted object as
            [xmin, ymin, xmax, ymax]
        gt_box (list of floats): location of ground truth object as
            [xmin, ymin, xmax, ymax]

    Returns:
        float: value of the IoU for the two boxes.

    Raises:
        AssertionError: if the box is obviously malformed
    """
    x1_t, x2_t, y1_t, y2_t = gt_box
    x1_p, x2_p, y1_p, y2_p = pred_box

    if (x1_p > x2_p) or (y1_p > y2_p):
        raise AssertionError(
            "Prediction box is malformed? pred box: {}".format(pred_box))
    if (x1_t > x2_t) or (y1_t > y2_t):
        raise AssertionError(
            "Ground Truth box is malformed? true box: {}".format(gt_box))

    if (x2_t < x1_p or x2_p < x1_t or y2_t < y1_p or y2_p < y1_t):
        return 0.0

    far_x = np.min([x2_t, x2_p])
    near_x = np.max([x1_t, x1_p])
    far_y = np.min([y2_t, y2_p])
    near_y = np.max([y1_t, y1_p])

    inter_area = (far_x - near_x + 1) * (far_y - near_y + 1)
    true_box_area = (x2_t - x1_t + 1) * (y2_t - y1_t + 1)
    pred_box_area = (x2_p - x1_p + 1) * (y2_p - y1_p + 1)
    iou = inter_area / (true_box_area + pred_box_area - inter_area)
    return iou


def get_single_image_results(gt_boxes, pred_boxes, iou_thr):
    """Calculates number of true_pos, false_pos, false_neg from single batch of boxes.

    Args:
        gt_boxes (list of list of floats): list of locations of ground truth
            objects as [xmin, ymin, xmax, ymax]
        pred_boxes (dict): dict of dicts of 'boxes' (formatted like `gt_boxes`)
            and 'scores'
        iou_thr (float): value of IoU to consider as threshold for a
            true prediction.

    Returns:
        dict: true positives (int), false positives (int), false negatives (int)
    """

    all_pred_indices = range(len(pred_boxes))
    all_gt_indices = range(len(gt_boxes))
    if len(all_pred_indices) == 0:
        tp = 0
        fp = 0
        fn = len(gt_boxes)
        return {'true_pos': tp, 'false_pos': fp, 'false_neg': fn}
    if len(all_gt_indices) == 0:
        tp = 0
        fp = len(pred_boxes)
        fn = 0
        return {'true_pos': tp, 'false_pos': fp, 'false_neg': fn}

    gt_idx_thr = []
    pred_idx_thr = []
    ious = []
    for ipb, pred_box in enumerate(pred_boxes):
        for igb, gt_box in enumerate(gt_boxes):
            iou = calc_iou_individual(pred_box, gt_box)
            if iou > iou_thr:
                gt_idx_thr.append(igb)
                pred_idx_thr.append(ipb)
                ious.append(iou)

    args_desc = np.argsort(ious)[::-1]
    if len(args_desc) == 0:
        # No matches
        tp = 0
        fp = len(pred_boxes)
        fn = len(gt_boxes)
    else:
        gt_match_idx = []
        pred_match_idx = []
        for idx in args_desc:
            gt_idx = gt_idx_thr[idx]
            pr_idx = pred_idx_thr[idx]
            # If the boxes are unmatched, add them to matches
            if (gt_idx not in gt_match_idx) and (pr_idx not in pred_match_idx):
                gt_match_idx.append(gt_idx)
                pred_match_idx.append(pr_idx)
        tp = len(gt_match_idx)
        fp = len(pred_boxes) - len(pred_match_idx)
        fn = len(gt_boxes) - len(gt_match_idx)

    return {'true_pos': tp, 'false_pos': fp, 'false_neg': fn}

In [3]:
def getIouScore(pred,grndt):
    x1_t,x2_t,y1_t,y2_t = grndt
    x1_p,x2_p,y1_p,y2_p = pred
    
    assert (x1_t < x2_t) and (y1_t < y2_t)
    
    assert (x1_p < x2_p) and (y1_p < y2_p)
    
    if x1_t > x2_p or x1_p > x2_t or y1_t > y2_p or y1_p > y2_t:
        return 0
    
    p1 = max(x1_t,x1_p)
    p2 = min(x2_t,x2_p)
    p3 = max(y1_t,y1_p)
    p4 = min(y2_t,y2_p)
    intersection = max((p2-p1) * (p4-p3),0)
    union = (x2_t-x1_t)*(y2_t-y1_t) + (x2_p-x1_p)*(y2_p-y1_p) - intersection
    return intersection/union


def getSingleImageResult(pred,grnd,threshold):
    if len(pred) ==0:
        tp = 0
        fp = 0
        fn = len(grnd)
        return {'true_pos': tp, 'false_pos': fp, 'false_neg': fn}
    if len(grnd) ==0:
        tp = 0
        fp = len(pred)
        fn = 0
        return {'true_pos': tp, 'false_pos': fp, 'false_neg': fn}
    
    iou = getIouScore(pred[0],grnd[0])
    if iou < threshold:
        tp = 0;
        fp = len(pred)
        fn =len(grnd)
    else:
        tp = 1
        fp = 0
        fn = 0
    return {'true_pos': tp, 'false_pos': fp, 'false_neg': fn}

def get_map_score(pred_info,ground_info):
    Precision =  []
    Recall = []
    key = sorted(pred_info.keys())
    threshold = np.linspace(0,1.0,11)
    for thrhld in threshold:
        Tp , Fp , Fn = 0,0,0
        for i in range(len(ground_info)):
            res = get_single_image_results([pred_info[key[i]]],[ground_info[key[i]]],thrhld)
            Tp+=res['true_pos']
            Fp+=res['false_pos']
            Fn+=res['false_neg']
        prec = Tp/(Tp+Fp)
        rec = Tp/(Tp+Fn)
        Precision.append(prec)
        Recall.append(rec)
    return np.mean(Precision) 
    